 #### <u>Lab 10</u>
##### CSCI316 - Big Data

**Group Members**
- Hunter Riddle (6451159)
- Oliver Small ()

#### Plain Tensorflow MLP Implementation

In [1]:
# imports
import numpy as np
import pandas as pd
import tensorflow as tf

/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/frame

In [2]:
# read in data
data = pd.read_csv("abalone_data.data", sep=',', header=None)
n_rows = data.shape[0]
n_columns = data.shape[1]

In [3]:
# format data
data.columns = ['sex', 'length', 'diameter', 'height', 'whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'rings']

# one hot encoding ('sex' attribute)
for (num, value) in enumerate(set(data['sex']), start=0):
    data['sex'] = data['sex'].replace([value], num)

In [4]:
# create testsets (holdout evaluation method, 70/30 split)
X_train = data.sample(int(n_rows*0.7))
X_test = data.drop(X_train.index)

# reset indexes
X_train = X_train.reset_index()
X_test = X_test.reset_index()

# cleaning up
y_train = np.array(X_train['rings'])
y_test = np.array(X_test['rings'])
X_train = np.array(X_train.iloc[: ,1:-1])
X_test = np.array(X_test.iloc[: ,1:-1])
print(X_train)

[[0.     0.45   0.34   ... 0.1605 0.0795 0.105 ]
 [1.     0.3    0.23   ... 0.056  0.0365 0.037 ]
 [0.     0.555  0.44   ... 0.4155 0.146  0.23  ]
 ...
 [0.     0.525  0.4    ... 0.3685 0.145  0.1735]
 [1.     0.345  0.255  ... 0.105  0.037  0.05  ]
 [0.     0.635  0.48   ... 0.413  0.228  0.36  ]]


In [5]:
# MLP hyperparameters
n_features = 8
n_neurons_hidden = 20
n_neurons_out = 1

In [6]:
# MLP variables
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
y_true = tf.placeholder(tf.float32, shape=(None), name="y")

W1 = tf.get_variable("weights1", dtype=tf.float32, initializer=tf.zeros((n_features, n_neurons_hidden)))
b1 = tf.get_variable("bias1", dtype=tf.float32, initializer=tf.zeros((n_neurons_hidden)))
W2 = tf.get_variable("weights2", dtype=tf.float32, initializer=tf.zeros((n_neurons_hidden, n_neurons_out)))
b2 = tf.get_variable("bias2", dtype=tf.float32, initializer=tf.zeros((n_neurons_out)))

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
# create network
h = tf.nn.relu(tf.matmul(X, W1)+b1)
z = tf.nn.softmax(tf.matmul(h, W2) + b2)

In [8]:
# define costs
y_hat = tf.exp(z) / tf.reduce_sum(tf.exp(z))
cross_entropy = -y_true * tf.log(y_hat)
cost = tf.reduce_mean(cross_entropy)

In [9]:
# training
learning_rate = 0.08
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(cost)

Instructions for updating:
Use tf.cast instead.


In [10]:
# execution
init = tf.global_variables_initializer()
n_epochs = 100
with tf.Session() as session:
    init.run()
    for epoch in range(n_epochs):
        session.run(training_op, feed_dict={
            X: X_train,
            y_true: y_train
        })
        
    predictions = session.run(z, feed_dict={'X:0':X_test})
    print(predictions[33:57])
   

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
